## The Model ##

\begin{aligned}
& \dot{S}_{i}=-\beta_{i} S_{i}(t) \frac{I_{i}(t)}{N_{i}(t)}+\frac{1}{\epsilon}\left[\sum_{j} \Phi_{j i} S_{j}(t)-\sum_{j} \Phi_{i j} S_{i}(t)\right] \\
& \dot{I}_{i}=\beta_{i} S_{i}(t) \frac{I_{i}(t)}{N_{i}(t)}-\gamma I_{i}(t)+\frac{1}{\epsilon}\left[\sum_{j} \Phi_{j i} I_{j}(t)-\sum_{j} \Phi_{i j} I_{i}(t)\right] \\
& \dot{R}_{i}=\gamma I_{i}(t)+\frac{1}{\epsilon}\left[\sum_{j} \Phi_{j i} R_{j}(t)-\sum_{j} \Phi_{i j} R_{i}(t)\right]
\end{aligned}

In [3]:
library(deSolve)

In [6]:
nodes <- read.csv("ITA_nodes.csv")
edges <- read.csv("ITA_edges.csv")

In [ ]:
## deprecated

SIR <- function(){
  beta <- 0.5
  gamma <- 0.2
  epsilon <- 1
  n <- length(nodes$population)
  N <- sum(nodes$population)
  times <- seq(0,200)
  param <- c(beta, gamma, epsilon)

  S<-nodes$population
  S[nodes$adm_name == "Padova"] <- S[nodes$adm_name == "Padova"] - 20
  I<-rep(0,n)
  I[nodes$adm_name == "Padova"] <- 20
  R<-rep(0,n)

  state<-c(S,I,R)

  odes <- function(t, state, param){
    with(as.list(c(param, state)), {
      S = matrix(state[1:n], nrow = n, ncol=1) ## state are patches
      I = matrix(state[(n+1):2*n], nrow = n, ncol=1)
      R = matrix(state[(2*n+1):3*n], nrow = n, ncol=1)

      beta <- beta
      gamma <- gamma
      epsilon <- epsilon

      dS <- numeric(legth(S)) ## You have to initialize them
      dI <- numeric(legth(I))
      dR <- numeric(legth(R))

      for (i in 1:n){ ## for da 1 a n per S,I,R
          dS[i] <- -beta * S[i] * (I[i] / N[i]) + (1 / epsilon)*(rowSums(mat[,i] * S[] - rowSums(mat[i,]*S[i])))
          dI[i] <- beta * S[i] * (I[i] / N[i]) - gamma * I[i] + (1 / epsilon)*(rowSums(mat[,i] * I[] - rowSums(mat[i,]*I[i])))
          dR[i] <- gamma * I[i] + (1 / epsilon)*(rowSums(mat[,i] * R[] - rowSums(mat[i,]*R[i])))
      }
      
      list(c(dS,dI,dR))
    })
  }

  mout <- lsoda (state, times, odes, param)

}


In [2]:
library(deSolve)
library(geosphere)

nodes <- read.csv("ITA_nodes.csv")
edges <- read.csv("ITA_edges.csv")

distancies <- matrix(0, nrow = nrow(nodes), ncol = nrow(nodes))

for (i in 1:nrow(nodes)) {
  for (j in 1:nrow(nodes)) {
    distancies[i, j] <- distHaversine(c(nodes$lon[i], nodes$lat[i]), c(nodes$lon[j], nodes$lat[j]))
  }
}

mat<-mat_rad/colSums(mat_rad)
beta<-0.5
gamma<-0.2
epsilon<-1
n<-length(nodes$population)
N<-sum(nodes$population)

odes <- function(t, state, param){
  with(as.list(c(param, state)), { #state sono i patches
    S = matrix(state[1:n], nrow = n, ncol=1)
    I = matrix(state[(n+1):(2*n)], nrow = n, ncol=1)
    R = matrix(state[(2*n+1):(3*n)], nrow = n, ncol=1)
    #print(sum(I)+sum(S)+sum(R))
    dI <- numeric(length(I))
    dS <- numeric(length(S))
    dR <- numeric(length(R))
    
    beta <- beta
    gamma <- gamma
    epsilon <- epsilon
    
    for(i in 1:n){
      dS[i] <- -beta*S[i]*I[i]/(S[i]+I[i]+R[i]) + 1/epsilon * (sum(mat[,i]*S[]) - sum(mat[i,]*S[i]))
      dI[i] <- beta*S[i]*I[i]/(S[i]+I[i]+R[i]) + 1/epsilon * (sum(mat[,i]*I[]) - sum(mat[i,]*I[i])) - gamma*I[i]
      dR[i] <- gamma*I[i] + 1/epsilon * (sum(mat[,i]*R[]) - sum(mat[i,]*R[i]))
    }
    list(c(dS,dI,dR))
  })
}

S<-nodes$population
S[nodes$adm_name=="Padova"]<-S[nodes$adm_name=="Padova"]-20
I<-rep(0,n)
I[nodes$adm_name=="Padova"]<-20
R<-rep(0,n)

state<-c(S,I,R)
param <- c(beta, gamma, epsilon)
times <- seq(1,200, by=1)
mout<-ode (state, times, odes, param)

mou <- as.data.frame(mout)
df<-data.frame(tempo=1, sus=sum(mou[1,2:n]), inf=sum(mou[1,(n+1):(2*n)]), rec=sum(mou[1,(2*n+1):(3*n)]))
for(t in 2:max(mou$time)){
  df <- rbind(df,data.frame(tempo=t, sus=sum(mou[t,2:n]), inf=sum(mou[t,(n+1):(2*n)]), rec=sum(mou[t,(2*n+1):(3*n)])))
}
p1_m <- ggplot(data=df) +
  theme_bw() + theme(legend.title = element_text(size = 2.5), legend.text = element_text(size = 5),legend.position="right") +
  scale_color_manual(values = c(
    'suscetible'= 'dodgerblue2',
    'infected' = 'red',
    'recovered' = 'green3')) +
  geom_line(aes(x=tempo, y=inf, colour="infected"), stat="identity") + geom_line(aes(x=tempo, y=rec, colour="recovered"), stat="identity") + geom_line(aes(x=tempo, y=sus, colour="suscetible"), stat="identity")+
  labs(color = '   ')+scale_fill_discrete(labels=c('R', 'S', 'I'))+
  ggtitle(paste("")) + ylab("Individuals") + xlab("Time")
print(p1_m)

ERROR: Error in eval(expr, envir, enclos): object 'mat_rad' not found
